In [1]:
import unittest

In [2]:
import sys
sys.path.append("/home/till/Desktop/GitHub/self-driving-car/")
sys.path.append("/home/till/Desktop/GitHub/self-driving-car/src")

In [3]:
from __future__ import annotations
from typing import Dict, List, Union

import logging
import os
import random
import sys
from collections import deque
from operator import itemgetter

import gym_donkeycar
import gymnasium as gym
import imageio
import ipywidgets as widgets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from IPython.display import display
from ipywidgets import HBox, VBox
from matplotlib import pyplot as plt
from PIL import Image
from ruamel.yaml import YAML
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from tensorboard import notebook
from tensorboard.backend.event_processing.event_accumulator import \
    EventAccumulator
from torch import distributions as dist
from torch.distributions import Categorical, Normal
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

import gymnasium as gym

# suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium.spaces.box") # module="gymnasium"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["IMAGEIO_IGNORE_WARNINGS"] = "True"

import stable_baselines3 as sb3
from gym_donkeycar.envs.donkey_env import DonkeyEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from stable_baselines3 import A2C, PPO, SAC
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

import src
from src.actor_critic_discrete import DiscreteActorCritic
from src.actor_critic_dreamer import ActorCriticDreamer
from src.actor_critic import ContinuousActorCritic
from src.blocks import CategoricalStraightThrough, ConvBlock
from src.categorical_vae import CategoricalVAE
from src.imagination_env import make_imagination_env
from src.mlp import MLP
from src.preprocessing import transform
from src.replay_buffer import ReplayBuffer
from src.rssm import RSSM
from src.utils import (load_config, make_env, save_image_and_reconstruction,
                       to_np, symlog, symexp, twohot_encode, ExponentialMovingAvg,
                       ActionExponentialMovingAvg, MetricsTracker)
from src.vae import VAE

torch.cuda.empty_cache()
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load the config
config = load_config()
for key in config:
    locals()[key] = config[key]

In [27]:
class TestAddition(unittest.TestCase):
    
    def addition(self,x,y):
        return x+y
    
    def test_addition(self):
        self.assertEqual(self.addition(1,2), 3)

In [19]:
# if __name__ == "__main__":
#     unittest.main()

hi


In [200]:
agent = src.actor_critic_discrete.DiscreteActorCritic()

Initializing critic.
Adding zero weight init to the output layer.

Initializing actor.


In [31]:
agent.critic

MLP(
  (layers): Sequential(
    (layer_0): Sequential(
      (0): Linear(in_features=1536, out_features=256, bias=True)
      (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (2): SiLU(inplace=True)
    )
    (layer_1): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (2): SiLU(inplace=True)
    )
    (layer_2): Sequential(
      (0): Linear(in_features=256, out_features=255, bias=True)
      (2): Softmax(dim=-1)
    )
  )
)

In [45]:
sample_input = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])

In [47]:
value_pred, critic_dist = agent.apply_critic(sample_input)

In [50]:
len(value_pred.shape)

0

In [57]:
critic_dist.shape

torch.Size([255])

In [59]:
config["num_buckets"]

255

In [63]:
torch.Size([config["num_buckets"]])

torch.Size([255])

In [51]:
class TestDiscreteActorCritic(unittest.TestCase):
    
    agent = src.actor_critic_discrete.DiscreteActorCritic()
    
    # test if the critic softmax sums to 1
    # test the critic output shape 
    # => for a single instance
    # => for a batch
    def test_critic(self):
        
        sample_input = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])
        value_pred, critic_dist = agent.apply_critic(sample_input)
        
        # value_pred should be a scalar without shape
        self.assertEqual(len(value_pred.shape), 0)

Initializing critic.
Adding zero weight init to the output layer.

Initializing actor.


In [65]:
sample_batch = torch.randn(32, config["H"] + config["Z"]).to(config["device"])
sample_batch.shape

torch.Size([32, 1536])

In [67]:
value_pred, critic_dist = agent.apply_critic(sample_batch)

In [72]:
torch.sum(critic_dist, dim=-1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0', grad_fn=<SumBackward1>)

In [8]:
rssm = RSSM().to(device)

Initializing encoder:
- adding ConvBlock((3, 64))                   ==> output shape: (64, 32, 32) ==> prod: 65536
- adding ConvBlock((64, 128))                   ==> output shape: (128, 16, 16) ==> prod: 32768
- adding ConvBlock((128, 256))                   ==> output shape: (256, 8, 8) ==> prod: 16384
- adding ConvBlock((256, 512))                   ==> output shape: (512, 4, 4) ==> prod: 8192
- adding ConvBlock((512, 256))                   ==> output shape: (256, 2, 2) ==> prod: 1024
- adding Flatten()
- adding Reshape: (*,1024) => (*,32,32)

Initializing decoder:
- adding Reshape: (*,1024) => (*,256,2,2)
- adding transpose ConvBlock(256, 256)                   ==> output shape: (256, 4, 4) ==> prod: 4096
- adding transpose ConvBlock(256, 512)                   ==> output shape: (512, 8, 8) ==> prod: 32768
- adding transpose ConvBlock(512, 256)                   ==> output shape: (256, 16, 16) ==> prod: 65536
- adding transpose ConvBlock(256, 128)                   ==> output shap

In [33]:
action = torch.randn(config["A"])
h = torch.randn(config["H"]).to(config["device"]).unsqueeze(0)
z = torch.randn(config["Z"]).to(config["device"]).unsqueeze(0)
rssm.step(action, h, z)

NameError: name 'rssm' is not defined

In [34]:
print("action:", action.shape)
print("h:", h.shape)
print("z:", z.shape)

action: torch.Size([3])
h: torch.Size([1, 512])
z: torch.Size([1, 1024])


In [61]:
action, log_prob, actor_entropy = agent.get_action(torch.randn(32, config["H"] + config["Z"]))

print("action:", action.shape)
print("log_prob:", log_prob.shape)
print("actor_entropy:", actor_entropy.shape)

action: torch.Size([32, 3])
log_prob: torch.Size([32])
actor_entropy: torch.Size([32])


In [46]:
action.shape

torch.Size([32, 3])

In [47]:
log_prob.shape

torch.Size([32])

In [48]:
actor_entropy.shape

torch.Size([32])

In [49]:
actor_entropy

tensor([5.6765, 5.6701, 5.6614, 5.6770, 5.6970, 5.6877, 5.6655, 5.7580, 5.7321,
        5.7102, 5.6841, 5.6886, 5.7618, 5.7470, 5.6534, 5.7194, 5.7698, 5.7119,
        5.6412, 5.7520, 5.7313, 5.6788, 5.6968, 5.7121, 5.6598, 5.6970, 5.6914,
        5.6608, 5.6692, 5.7146, 5.7430, 5.7094], device='cuda:0',
       grad_fn=<SumBackward1>)

In [ ]:
"""
Tests:
- output shapes
- that the action is within the valid action range
- that the log probability and entropy are tensors
"""
sample_instance = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])

action, log_prob, entropy = self.test_agent.get_action(sample_instance)

# action should have shape (n_actions,)
self.assertEqual(action.shape, torch.Size([self.test_agent.n_actions]))

# Check that the action is within the valid action range
self.assertTrue(torch.all(action >= self.test_agent.action_buckets[0]))
self.assertTrue(torch.all(action <= self.test_agent.action_buckets[-1]))

# log_prob and entropy should be tensors
self.assertTrue(isinstance(log_prob, torch.Tensor))
self.assertTrue(isinstance(entropy, torch.Tensor))

In [210]:
sample_instance = torch.randn(config["H"] + config["Z"]).to(config["device"]) # torch.Size([1536])
sample_batch = torch.randn(32, config["H"] + config["Z"]).to(config["device"]) # torch.Size([32, 1536])

print("sample instance:", sample_instance.shape)
print("sample batch:", sample_batch.shape)

sample instance: torch.Size([1536])
sample batch: torch.Size([32, 1536])


In [211]:
action, log_prob, actor_entropy = agent.get_action(sample_instance)

print("action:", action.shape)
print("log_prob:", log_prob.shape)
print("actor_entropy:", actor_entropy.shape)

assert action.shape == torch.Size([1, config["A"]])
assert log_prob.shape == torch.Size([1])
assert actor_entropy.shape == torch.Size([1])

action: torch.Size([1, 3])
log_prob: torch.Size([1])
actor_entropy: torch.Size([1])


In [212]:
action, log_prob, actor_entropy = agent.get_action(sample_batch)

print("action:", action.shape)
print("log_prob:", log_prob.shape)
print("actor_entropy:", actor_entropy.shape)

assert action.shape == torch.Size([32, config["A"]])
assert log_prob.shape == torch.Size([32])
assert actor_entropy.shape == torch.Size([32])

action: torch.Size([32, 3])
log_prob: torch.Size([32])
actor_entropy: torch.Size([32])


In [213]:
value_pred, critic_dist = agent.apply_critic(sample_instance)
print("value pred:", value_pred.shape)
print("critic dist:", critic_dist.shape)

value pred: torch.Size([1])
critic dist: torch.Size([1, 255])


In [214]:
value_pred, critic_dist = agent.apply_critic(sample_batch)
print("value pred:", value_pred.shape)
print("critic dist:", critic_dist.shape)

value pred: torch.Size([32])
critic dist: torch.Size([32, 255])


In [196]:
torch.ones([32]).shape

torch.Size([32])

In [ ]:
# todo: add proper vector env

### envs = gym.vector.AsyncVectorEnv(
###     [
###         lambda: gym.make(
###             "LunarLander-v2",
###             gravity=np.clip(
###                 np.random.normal(loc=-10.0, scale=1.0), a_min=-11.99, a_max=-0.01
###             ),
###             enable_wind=np.random.choice([True, False]),
###             wind_power=np.clip(
###                 np.random.normal(loc=15.0, scale=1.0), a_min=0.01, a_max=19.99
###             ),
###             turbulence_power=np.clip(
###                 np.random.normal(loc=1.5, scale=0.5), a_min=0.01, a_max=1.99
###             ),
###             max_episode_steps=600,
###         )
###         for i in range(n_envs)
###     ]
### )

In [167]:
env = make_env()

Making a toy env.
Making 3 vectorized envs.
Adding a Gymnasium RecordEpisodeStatistics wrapper.
Adding a TimeLimit wrapper with 1000 max episode steps.
Adding an AutoReset wrapper.
Adding a RescaleActionV0 wrapper. Low: [-1. -1. -1.], High: [1. 1. 1.]


In [170]:
obs, info = env.reset()
obs.shape # not working vectorized!

(96, 96, 3)

In [107]:
ema = ActionExponentialMovingAvg()

In [63]:
log_prob

tensor([-6.3694, -6.6164, -5.3328, -5.6937, -6.3013, -5.5463, -5.1594, -5.9310,
        -6.3248, -5.9097, -5.2662, -5.4119, -5.9785, -5.9952, -5.9080, -5.6857,
        -4.8114, -5.9112, -5.5444, -5.0474, -5.6220, -5.6623, -5.8521, -6.3300,
        -5.0737, -5.4063, -4.8265, -5.4533, -5.7005, -5.8377, -6.2331, -5.2692],
       device='cuda:0', grad_fn=<SumBackward1>)

In [67]:
np.log(0.0000000001)

-23.025850929940457

In [68]:
actor_entropy

tensor([5.7355, 5.5546, 5.7163, 5.6698, 5.6928, 5.6896, 5.7400, 5.6838, 5.6797,
        5.6873, 5.7251, 5.6763, 5.7283, 5.6503, 5.7192, 5.7047, 5.5833, 5.6886,
        5.7189, 5.7330, 5.7141, 5.6925, 5.6848, 5.7214, 5.7266, 5.6649, 5.6694,
        5.7534, 5.6803, 5.5994, 5.7590, 5.6697], device='cuda:0',
       grad_fn=<SumBackward1>)

In [116]:
agent = src.actor_critic_discrete.DiscreteActorCritic()

Initializing critic.
Adding zero weight init to the output layer.

Initializing actor.


In [122]:
sample_instance = torch.randn(config["H"] + config["Z"]).to(config["device"])  # torch.Size([1536])#
sample_instance.shape

torch.Size([1536])

In [133]:
agent.get_action(to_np(sample_instance))

(tensor([[ 0.4231, -0.1996,  0.1345]], device='cuda:0'),
 tensor([-6.4722], device='cuda:0', grad_fn=<SumBackward1>),
 tensor([1.8924], device='cuda:0', grad_fn=<MeanBackward1>))